# Packages

In [ ]:
!pip install seaborn
!pip install causalgraphicalmodels
!pip install matplotlib
!pip install -U scikit-learn
!pip install --upgrade tensorflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
import causalgraphicalmodels
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import tensorflow as tf
from sklearn.metrics import r2_score

     |████████████████████████████████| 22.3MB 1.5MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
Requirement already up-to-date: tensorflow in /usr/local/lib/python3.7/dist-packages (2.5.0)


# Model Building

In [ ]:
def sigmoid(x):
  return (1 / (1 + math.exp(-x)))
def epsilon_x ():
    return np.random.normal(loc = 0,scale = 0.5)
def epsilon_y ():
    return np.random.normal(loc = 0 , scale = 1)
def epsilon_z ():
    return np.random.normal(loc=-1,scale=1)
def para_c1():
    return np.random.normal(loc=-2,scale=1)
def para_c2():
    return np.random.normal(loc=-2,scale = 1)
def para_cy():
    return np.random.normal(loc=1,scale=1)

def build_x(u):
    return np.random.binomial(n=1,p=sigmoid(u + epsilon_x()))

def build_causal_model(n_samples,d):
    scm_fd = np.empty((n_samples,d+2))
    col_name = list()
    for i in range(n_samples):
        u1 = np.random.normal(loc=-2 ,scale = 1)
        x = build_x(u1)
        scm_fd[i][0] = x
        if i == 0:
          col_name.append('x')
        z_sum = 0.0
        for j in range(1,d+1,1):
            p=sigmoid(para_c1() + para_c2() *x + epsilon_z())
            z = np.random.binomial(n=1,p=p)
            scm_fd[i][j] = z
            p = para_cy()
            z_sum += z*p
            if i==0:
              col_name.append('z'+str(j-1))
        y = sigmoid(2*z_sum + u1 + epsilon_y())
        scm_fd[i][d+1] = y
        if i == 0:
          col_name.append('y')
    return scm_fd,col_name

# Ground Truth (Do Calculus)

In [ ]:
def causal_do(df,new_x):
    scm_copy = df.copy()
    scm_copy[:,0] = new_x
    for i in range(scm_copy.shape[0]):
        u1 = np.random.normal(loc = -2, scale = 1)
        x = new_x[i]
        #print(x)
        z_sum = 0.0
        d = scm_copy.shape[1] - 2
        for j in range(1,d+1,1):
            p=sigmoid(para_c1() + para_c2() *x + epsilon_z())
            z = np.random.binomial(n=1,p=p)
            scm_copy[i][j] = z
            z_sum += z*para_cy()
        y = sigmoid(2*z_sum + u1 + epsilon_y())
        scm_copy[i][d+1] = y

    print('modified')
    print(scm_copy)

    return scm_copy

**Estimate E[Y|X=1] - E[Y|X=0]**
from a dataframe `df` of samples.

  

In [ ]:
def estimate_conditional_expectation1(df):
    scm_cols = np.size(df,axis=1)
    scm_rows = np.size(df,axis=0)
    a = df[:,0] == 0
    b = df[:,0] == 1
    y = df[:,df.shape[1]-1]

    p = np.mean(y[b])
    q = np.mean(y[a])
    delta = p-q
    return p,q,delta
def ab_test1(scm, n):
    n_a = int(n / 2)
    n_b = n - n_a
    set_variable = np.array([0]*n_a + [1]*n_b)
    scm = causal_do(scm,set_variable)
    # print('modified->',scm)
    return estimate_conditional_expectation1(scm)


# ATE using CWO

**Weight Generattion**

In [ ]:
def estimate_p_z_x(df):
  d = df.shape[1]-3
  df_copy = df.copy()
  df_size = df.shape[0]
  proba_z_x = np.ones(df.shape[0])
  X=df['x'].values.reshape(-1, 1)

  for i in range(d):

    y=df[('z'+str(i))]
    #print(i , y)
    clf = LogisticRegression(random_state=0).fit(X, y)
    predict = clf.predict_proba(X)
    for j in range(df_size):
      proba_z_x[j] *= predict[j,int(y[j])]
    del clf

    #print(proba_z_x)
  prob_frame = pd.DataFrame(proba_z_x , columns = ["z|x"])
  #print(prob_frame)

  return prob_frame


def prob_z(data):
  d=data.shape[1]-3
  z=data.iloc[:,1:1+d]
  # d = z.shape[1]
  l=z.shape[0]
  p={}
  for i in range(d):
    p[i]={}
    prob = z['z'+str(i)] == 0
    p[i][0]= prob.mean()
    prob = z['z'+str(i)] == 1
    p[i][1]= prob.mean()
  pz=np.ones(l)
  for i in range(l):
    for j in range(d):
      pz[i]*=p[j][z.loc[i][j]]
  z_hat = pd.DataFrame(pz,columns =['z_hat'])
  return z_hat


def weightGenerator(df):
  data=df.copy()
  data1 = estimate_p_z_x(data).to_numpy()
  data2 = prob_z(data).to_numpy()
  data_prob = np.divide(data2,data1)
  return data_prob

In [ ]:
def cwoBeta(X,y,weight,beta):
  Beta1 = LinearRegression().fit(X,y,sample_weight=weight)
  test_x=np.array([0, 1]).reshape((-1, 1))
  if beta==1:
    y_pred=Beta1.predict(test_x)
  else:
    y_pred=Beta1.predict(X)
  return y_pred

In [ ]:
def weightedATE(df):
  data=df.copy()
  d=data.shape[1]-3
  Z=data.iloc[:,1:1+d].values.reshape(-1, d)
  X=data['x'].values.reshape(-1, 1)
  Y=data['y'].values.reshape(-1, 1)
  data['weight'] = weightGenerator(data)
  sampleWeight= data['weight']
  y_pred=cwoBeta(Z,Y,sampleWeight,2)
  sampleWeight2 =np.ones((df.shape[0],), dtype=int)
  ate=cwoBeta(X,y_pred,sampleWeight2,1)
  return ate[1],ate[0],ate[1]-ate[0]


# NeuralNetwork ATE

In [ ]:
#low d

hpDict={'conv_0_units': 10,
 'conv_1_units': 10,
 'dropout': 0.1,
 'dropout_0_': 0.1,
 'dropout_1_': 0.1,
 'input_units': 80,
 'learning_rate': 0.0006794707285250219,
 'n_layers': 2}

In [ ]:
#high d

hpDict={'conv_0_units': 70,
 'conv_1_units': 80,
 'conv_2_units': 10,
 'dropout': 0,
 'dropout_0_': 0.2,
 'dropout_1_': 0.10000000000000004,
 'dropout_2_': 0.1,
 'input_units': 90,
 'learning_rate': 0.0003239508297196215,
 'n_layers': 3}

In [ ]:
def nnBeta(X,y,weight,hp):
  d=X.shape[1]
  Beta2 = tf.keras.models.Sequential()
  Beta2.add(tf.keras.layers.Dense(hp['input_units'],activation='relu',input_shape=(d,)))
  Beta2.add(tf.keras.layers.Dropout(hp['dropout']))
  for i in range(hp['n_layers']):
    Beta2.add(tf.keras.layers.Dense(hp[f'conv_{i}_units'], activation='relu'))
    Beta2.add(tf.keras.layers.Dropout(hp[f'dropout_{i}_']))

  Beta2.add(tf.keras.layers.Dense(1, activation='linear'))
  Beta2.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=hp['learning_rate']), loss='mse')

  early_stop=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=20, verbose=1, mode='auto', restore_best_weights=True)
  history=Beta2.fit(X,y,epochs=1000,
                    validation_split=0.2,
                    shuffle = True,
                    callbacks=[early_stop],
                    verbose=0
                    ,sample_weight=weight
                    )
  y_pred = Beta2.predict(X)
  return y_pred

In [ ]:
def NeuralATE(df,hp):
  data=df.copy()
  d=data.shape[1]-2
  Z=data.iloc[:,1:d+1].values.reshape(-1, d)
  X=data['x'].values.reshape(-1, 1)
  Y=data['y'].values.reshape(-1, 1)
  sampleWeight = weightGenerator(data)
  y_pred = nnBeta(Z,Y,sampleWeight,hp)
  sampleWeight2 =np.ones((df.shape[0],), dtype=int)
  ate=cwoBeta(X,y_pred,sampleWeight2,1)
  return ate[1][0],ate[0][0],ate[1][0]-ate[0][0]


# mu and Dataframe saving

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My Drive/
%cd Datasets

[Errno 2] No such file or directory: 'drive/My Drive/'
/content/drive/My Drive


In [ ]:
D=6

In [ ]:
def doDo(d):
  N=10000000
  fdDo,col_name=build_causal_model(N,d)
  mu1,mu0,muATE=ab_test1(fdDo, N)
  df = pd.DataFrame(fdDo,columns = col_name)
  return df,mu1,mu0

In [ ]:
fd,mu1,mu0=doDo(D)
print("Estimated ATE: {:.3f}".format(mu1-mu0))
print("mu(1): {:.3f}".format(mu1))
print("mu(0): {:.3f}".format(mu0))

modified
[[0.         0.         0.         ... 0.         0.         0.09138255]
 [0.         0.         0.         ... 0.         0.         0.01437216]
 [0.         0.         0.         ... 0.         0.         0.06594632]
 ...
 [1.         0.         0.         ... 0.         0.         0.15628845]
 [1.         0.         0.         ... 0.         0.         0.0598134 ]
 [1.         0.         0.         ... 0.         0.         0.10109322]]
Estimated ATE: -0.116
mu(1): 0.229
mu(0): 0.344


In [ ]:
fd.to_pickle("drive/MyDrive/Datasets/f_dataframe_6.pkl")

In [ ]:
string1 = str(mu1) + " " + str(mu0)
print(string1)
f = open("drive/MyDrive/Datasets/f_mu_6.txt","w")
f.write(string1)
f.close()

0.22851276750896699 0.3444887749881229


# Result Generation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
fd = pd.read_pickle("drive/MyDrive/Datasets/f_dataframe_10.pkl")
print(fd)
f = open('drive/MyDrive/Datasets/f_mu_10.txt','r')
mm = f.read()
mu1,mu0 = mm.split()
mu1=float(mu1)
mu0=float(mu0)
print(mu1,mu0)

           x   z0   z1   z2   z3   z4   z5   z6   z7   z8   z9         y
0        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.120565
1        0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.999094
2        1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.232915
3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.033733
4        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.218860
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...       ...
9999995  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.102548
9999996  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.184443
9999997  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.242544
9999998  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.321488
9999999  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.242391

[10000000 rows x 12 columns]
0.25690356479199317 0.4349996790635731


In [ ]:
def MAAEgenerator(fd,mu1,mu0):
  muN1,muN0,_=NeuralATE(fd,hpDict)
  muC1,muC0,_=weightedATE(fd)
  maaeN=(abs(muN1-mu1)+abs(muN0-mu0))/2
  maaeC=(abs(muC1-mu1)+abs(muC0-mu0))/2
  return maaeN,maaeC


In [ ]:
N_SAMPLE=2000

In [ ]:
muN=np.zeros(100)
muC=np.zeros(100)
for k in range(100):
  print("K= ",k)
  db=fd.sample(n=N_SAMPLE).reset_index(drop=True)
  muN[k],muC[k]=MAAEgenerator(db,mu1,mu0)
  print(k,muN[k],muC[k])
MAAE_N=np.median(muN)
print("MAAE NN",MAAE_N)
MAAE_C=np.median(muC)
print("MAAE CWO",MAAE_C)
print("MAAE NN: {:.5f}".format(MAAE_N))
print("MAAE CWO: {:.5f}".format(MAAE_C))

K=  0
Restoring model weights from the end of the best epoch.
Epoch 00036: early stopping
0 0.014967302938202226 0.02036279517435255
K=  1
Restoring model weights from the end of the best epoch.
Epoch 00030: early stopping
1 0.005050455816114463 0.014693580910319165
K=  2
Restoring model weights from the end of the best epoch.
Epoch 00029: early stopping
2 0.005337448987116772 0.015479276413675164
K=  3
Restoring model weights from the end of the best epoch.
Epoch 00031: early stopping
3 0.01991343083251254 0.018599353887328585
K=  4
Restoring model weights from the end of the best epoch.
Epoch 00035: early stopping
4 0.024148787391648413 0.01979394493596251
K=  5
Restoring model weights from the end of the best epoch.
Epoch 00045: early stopping
5 0.015096315566239138 0.015705890843967896
K=  6
Restoring model weights from the end of the best epoch.
Epoch 00040: early stopping
6 0.01721391216548568 0.02566606315579048
K=  7
Restoring model weights from the end of the best epoch.
Epoch